In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
from datetime import datetime
import numpy as np
import pickle

In [5]:
url = 'https://page.kakao.com'
options = Options()
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36")
data_list = []

driver = webdriver.Chrome()
driver.get(url)
time.sleep(3)

driver.execute_script(f"document.querySelector('button.pr-16pxr').click();")
time.sleep(3)

id_input = driver.find_element(By.CSS_SELECTOR, '#loginId--1')
id_input.send_keys('tkdehdjajsl@gmail.com')
pw_input = driver.find_element(By.CSS_SELECTOR, '#password--2')
pw_input.send_keys('gksruf9507')
time.sleep(5)

submit_btn = driver.find_element(By.CSS_SELECTOR, 'button.submit')
submit_btn.click()
time.sleep(5)

# 로그인 후 쿠키 저장
cookies = driver.get_cookies()
with open('kakao_cookies.pkl', 'wb') as f:
  pickle.dump(cookies, f)

driver.quit()

In [6]:
url = 'https://page.kakao.com'
options = Options()
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36")
data_list = []

# 여기에 로그인 파트

driver = webdriver.Chrome(options=options)

driver.get(url)

# 쿠키 로드
with open('kakao_cookies.pkl', 'rb') as f:
  cookies = pickle.load(f)
  for cookie in cookies:
    driver.add_cookie(cookie)

driver.refresh()

time.sleep(3)

df = pd.read_csv('Links.csv')
for url in df['link'].iloc[:2001]:
  data = {}

  try:
    # 두번째 페이지 먼저 호출
    driver.get(url+'?tab_type=about')
    time.sleep(3)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
      
    # JavaScript를 사용하여 버튼이 존재하면 클릭
    script = f"""
    var button = document.querySelector('button.font-medium1.flex.h-59pxr.cursor-pointer.items-center.justify-center.border-0.border-t-1.border-solid.border-line-10.px-24pxr.text-el-50');
    if (button) {{
        button.click();
    }}
    """
    driver.execute_script(script)
    time.sleep(1)

    # 키워드 정보가 없거나, 발행자 정보가 한개 이상이거나, 또는 전자책 정가 정보가 없으면 패스
    keywords = soup.select('div.flex.w-full.flex-wrap.px-18pxr.pb-10pxr > a > div > span')
    if not keywords:
      continue

    others = soup.select('div.flex.w-full.flex-col.items-start.px-18pxr.pb-18pxr > div')[2:]
    if len(others) != 3:
      continue

    check = others[0].select('span.break-all.align-middle')
    if check:
      continue

    title = soup.select_one('span.font-large3-bold.mb-3pxr.text-ellipsis.break-all.text-el-70.line-clamp-2').text
    if '단행본' in title:
      continue
    else:
      data['title'] = title
      
    # 첫번째 페이지 호출
    driver.get(url)

    time.sleep(2)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
      
    for keyword in keywords:
      data['keywords'] = ''.join(keyword.text for keyword in keywords)

    data['publisher'] = others[0].select('span')[1].text
    data['users'] = others[1].select('span')[1].text
    data['price'] = others[2].select('span')[1].text

    author = soup.select_one('span.font-small2.mb-6pxr.text-ellipsis.text-el-70.opacity-70.break-word-anywhere.line-clamp-2').text
    data['author'] = author
    ing = soup.select_one('div.mt-6pxr.flex.items-center > span.font-small2.text-el-70.opacity-70').text
    data['ing'] = ing
    genre = soup.select('span.break-all.align-middle')[1].text
    data['genre'] = genre
    view = soup.select('span.text-el-70.opacity-70')[1].text
    data['viewCount'] = view
    score = soup.select('span.text-el-70.opacity-70')[2].text
    data['score'] = score
    counts = soup.select('span.text-ellipsis.break-all.line-clamp-1.font-small2-bold.text-el-70')
    uploaded = counts[0].text
    data['uploaded'] = uploaded
    review = counts[1].text
    data['reviewCount'] = review
    date = soup.select_one('div.flex.flex-col.pr-14pxr > div > span.break-all.align-middle').text
    data['date'] = date

    free_info = soup.select_one('div.flex.flex-wrap.text-el-70 > span.opacity-70.font-small2').text
    data['free_info'] = free_info
    
    # 필요한 모든 키가 존재하는지 확인
    required_keys = ['title', 'keywords', 'publisher', 'users', 'price', 'author', 'ing', 'genre', 'viewCount', 'score', 'uploaded', 'reviewCount', 'date', 'free_info']
    if all(key in data for key in required_keys):
      data_list.append(data)
  except:
    continue

df1 = pd.DataFrame(data_list)
df1.to_csv('Data1-2000.csv', index=False, encoding='utf-8-sig')

driver.quit()

IndexError: list index out of range

In [3]:
df = pd.read_csv('ExampleData.csv')
df

,title,keywords,publisher,users,price,author,ing,genre,viewCount,score,uploaded,reviewCount,date,free_info
0,내 머릿속 소드마스터,#정통#성장,로크미디어,전체이용가,100원/회차 당,괴도고구마,완결,판타지,"1,515",1.0,전체 152,전체 0,24.06.27,25장
1,퇴장한 악녀의 브런치 카페,#로맨스판타지#초월적존재#영혼체인지#왕족/귀족#동거물#오해물#첫사랑#능력남#다정남#...,코핀커뮤니케이션즈,전체이용가,100원/회차 당,지미신,월~금 연재,로판,19.2만,9.7,전체 85,전체 312,24.06.27,0장
2,양치기 소년에게 납치당했습니다,#로맨스판타지#신화물#운명#성장물#다정남#왕족/귀족#대형견남#능력녀#걸크러쉬#계략녀...,다소다미디어,전체이용가,100원/회차 당,멜리타,완결,로판,139,완결,전체 149,전체 0,24.06.27,4장
3,벤츠 타는 마법사,#차원이동#마법사,라온이앤엠,전체이용가,100원/회차 당,정주(丁柱),완결,판타지,"5,663",7.5,전체 161,전체 8,24.06.27,26장
4,무한 회귀로 천마가 되다,#회귀#천재#노력#천마#무공,현나라,전체이용가,100원/회차 당,제허니,연재,판타지,1.2만,9.8,전체 56,전체 3,24.06.27,15장
5,몸 잘 쓰는 천만 배우,#성장#노력#배우,로크미디어,전체이용가,100원/회차 당,이새우,완결,현판,"3,183",10.0,전체 175,전체 0,24.06.27,25장
6,결혼까지 할 줄은 몰랐어 [19세 완전판][외전 독점],#현대로맨스#시월드#가족물#운명#다정남#소유욕/독점욕#순정남#재벌남#카리스마남#존댓...,래이니북스,19세이용가,100원/회차 당,다온향,연재,로맨스,640,10.0,전체 9,전체 0,24.06.27,0장
